## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])
#sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/computeStats.py'])
from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSSSBSBS'
meas='SNOW'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
135
+-------------------+-------------------+-------------------+-------------------+--------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------+---------+------+--------------------+------+
|            coeff_1|            coeff_2|            coeff_3|            coeff_4|             coeff_5|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|             res_4|             res_5|         res_mean|    station|total_var|undefs|              vector|  year|
+-------------------+-------------------+-------------------+-------------------+--------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------+---

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'coeff_5', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00013035,1958.0,-0.413334
1,USC00085793,1958.0,-0.413334
2,USC00010583,1958.0,-0.413334
3,USC00082220,1958.0,-0.413334
4,USC00013255,1958.0,-0.401763


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.tail(10)

station,USC00010252,USC00010402,USC00010407,USC00010583,USC00011084,USC00011566,USC00011685,USC00011803,USC00011807,USC00012577,...,USW00013838,USW00013846,USW00013858,USW00013885,USW00013894,USW00013899,USW00093826,USW00093828,USW00093841,USW00093859
year,,,,,,,,,,,,,,,,,,,,,
1969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973.0,NaN,NaN,NaN,1.628479,151.593994,NaN,NaN,0.749563,75.609272,NaN,...,NaN,NaN,NaN,NaN,1.224178,3.432550,NaN,NaN,0.521045,NaN
1977.0,NaN,NaN,-0.726102,-0.611385,NaN,NaN,NaN,-0.386022,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.576655,-0.599756,NaN,NaN,-0.771703,NaN
1988.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.381792,NaN,NaN,NaN,NaN,NaN
1989.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993.0,-0.720244,-0.491137,NaN,-0.547847,-0.386792,-0.411914,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.500800,NaN,NaN,NaN,NaN,NaN
1994.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1995.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996.0,NaN,-0.424198,NaN,-0.428977,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.399346,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat): #Root Mean Squared
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  28.7632185848
RMS removing mean-by-station=  23.0209139383
RMS removing mean-by-year   =  23.5744424912


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 28.7632185848
0 after removing mean by year    = 23.5744424912
0 after removing mean by stations= 18.9318329054
1 after removing mean by year    = 18.4156178653
1 after removing mean by stations= 18.213670902
2 after removing mean by year    = 18.1292592506
2 after removing mean by stations= 18.0920993213
3 after removing mean by year    = 18.0749382916
3 after removing mean by stations= 18.0666460711
4 after removing mean by year    = 18.0624696332
4 after removing mean by stations= 18.0602884597
